In [1]:
#Update your token
STUDENT_TOKEN = 'EUGENIOMARCHIORI'

Server communication game and game_loop.

In [2]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'

  def get_board(self):
    return json.loads(self.state)

  def actions(self):
    return []

  def print_game(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    print('\033[92mCreating new game...\033[0m')
    id = new_game(game_type, multi_player)

  print('\033[92mJoining game with id: %s\033[0m' % id)
  player = join_game(game_type, id)

  print('\033[92mPlaying as %s\033[0m' % player)

  game = game_state(game_type, id, GameClass)
  print('\033[91mWaiting for the other player to join...\033[0m')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('\033[94mdraw\033[0m')
      else:
        print('\033[92mwinner\033[0m' if game.player == player else '\033[91mloser\033[0m')
      return
    if game.player == player:
      print('Making next move...')
      move = solver(game)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

Random solver

In [3]:
def random_solver(game):
  actions = game.actions()
  return actions[0]


Game specific code

In [4]:
from functools import reduce
from copy import copy, deepcopy
import json
import random

class Domineering(Game):
  def __init__(self, state, status, player):
    Game.__init__(self, state, status, player)

  def actions(self):
    actions = []
    board = self.get_board()
    player = self.player

    for row in range(len(board)):
      for col in range(len(board[row])):
        if board[row][col] == '.':
          if player == 'H' and col + 1 < len(board[row]) - 1 and board[row][col + 1] == '.':
            actions.append((row, col))
          elif player == 'V' and row + 1 < len(board) and board[row + 1][col] == '.':
            actions.append((row, col))
    return actions

  def print_game(self):
    for row in self.get_board():
      print(row)

  def other_player(self):
    if self.player == 'V': return 'H'
    if self.player == 'H': return 'V'

In [5]:
game_loop(random_solver, Domineering, 'domineering', multi_player=False, id=None)

Creating new game...
b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>\n'
b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>\n'
Joining game with id: 119
Playing as V
b'{"Message": "Error getting game, please retry"}'
Waiting for the other player to join...
['.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
Making next move...
b'Valid move'
['V', '.', '.', '.', '.', '.']
['